In [1]:
import sys
sys.path.append("../input/rank-gauss")
from gauss_rank_scaler import GaussRankScaler

In [2]:
!pip install ../input/pytorchtabnetpretraining/pytorch_tabnet-2.0.1-py3-none-any.whl

Processing /kaggle/input/pytorchtabnetpretraining/pytorch_tabnet-2.0.1-py3-none-any.whl


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.loss import _WeightedLoss

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import warnings
warnings.filterwarnings('ignore')

In [4]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
drug_ids = pd.read_csv('../input/lish-moa/train_drug.csv')
train_targets_noscore = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
folds = np.load('../input/fold-save/folds.npy')
train_targets_pl = pd.read_csv('../input/sin-submission/submission.csv')
train_features_pl = pd.read_csv('../input/test-public-moa/test_features.csv')

ss = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [5]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [6]:
nfolds = 7
nstarts = 3
nepochs = 200
fold_seed = 42
batch_size = 128
val_batch_size = batch_size * 4
ntargets = 206
# targets = [col for col in train_targets.columns]
val_criterion = nn.BCEWithLogitsLoss()
trn_criterion = SmoothBCEwLogits(smoothing=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
genes_comp = 80
cells_comp = 10

In [7]:
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [8]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

train = preprocess(train_features)
test = preprocess(test_features)
train_pl = preprocess(train_features_pl)


In [9]:
def quantile(tr, te, feats):
    data_all = pd.concat([tr, te], ignore_index = True)
    scaler = GaussRankScaler()
    data_all[feats] = scaler.fit_transform(data_all[feats])
    
    tr = data_all[:len(tr)]
    te = data_all[len(tr):]
    
    return tr, te

_, train_pl = quantile(train, train_pl, GENES)
_, train_pl = quantile(_, train_pl, CELLS)
train, test = quantile(train, test, GENES)
train, test = quantile(train, test, CELLS)

In [10]:
def pca(tr, te, feats, n_components, group):
    data_all = pd.concat([tr, te])
    pca_transformer = PCA(n_components=n_components, random_state=42)
    data_pca = pca_transformer.fit_transform(data_all[feats])
    
    if group == 'genes':
        cols = [f'pca_G-{i}' for i in range(n_components)]
    elif group == 'cells':
        cols = [f'pca_C-{i}' for i in range(n_components)]
    
    data_all = pd.concat([data_all, pd.DataFrame(data_pca, columns=cols)], axis=1)
    
    tr = data_all[:len(tr)]
    te = data_all[len(tr):]
    
    return tr, te

_, train_pl = pca(train, train_pl, GENES, genes_comp, 'genes')
_, train_pl = pca(_, train_pl, CELLS, cells_comp, 'cells')
train, test = pca(train, test, GENES, genes_comp, 'genes')
train, test = pca(train, test, CELLS, cells_comp, 'cells')

In [11]:
targets = train_targets.columns[1:]
train_targets = train_targets.merge(drug_ids, on='sig_id', how='left')

In [12]:
train_pl = train_pl.reset_index(drop=True)

In [13]:
train_targets = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
train_targets_pl = train_targets_pl.loc[train_pl['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)
train_pl = train_pl.loc[train_pl['cp_type']==0].reset_index(drop=True)

In [14]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [15]:
train_targets['fold'] = folds
train_targets.drug_id.value_counts().value_counts().sort_index()

1       129
2         3
3         2
4         3
5        66
6      2774
7       196
8         4
11        4
12       64
13       25
14        6
18        3
19        1
178       1
186       1
194       1
196       1
202       1
203       1
246       1
718       1
Name: drug_id, dtype: int64

In [16]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

In [17]:
top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]
print(len(top_feats))

785


In [18]:
# add pca feature indices to feature indices
pca_feats = [i for i in range(875, 875+genes_comp+cells_comp)]
top_feats.extend(pca_feats)

In [19]:
folds = train_targets.fold.values

In [20]:
train = train.values[:, top_feats]
train_pl = train_pl.values[:, top_feats]
test = test.values[:, top_feats]
train_targets = train_targets[targets].values
train_targets_pl = train_targets_pl[targets].values

In [21]:
oof = np.zeros((len(train), nstarts, ntargets))
oof_targets = np.zeros((len(train), ntargets))
preds = np.zeros((len(test), ntargets))

In [22]:
def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(targets):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [23]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

In [24]:
train_targets_pl

array([[0.00142804, 0.00172544, 0.00309356, ..., 0.00229157, 0.00430772,
        0.00202517],
       [0.00099383, 0.00179054, 0.00308135, ..., 0.00225782, 0.00113765,
        0.00293713],
       [0.00137616, 0.00148251, 0.0024345 , ..., 0.0022994 , 0.00094954,
        0.00226204],
       ...,
       [0.00188603, 0.00172753, 0.00144775, ..., 0.0019423 , 0.00076604,
        0.00144087],
       [0.00223348, 0.00146616, 0.00178669, ..., 0.00260949, 0.00093238,
        0.00307839],
       [0.00124144, 0.00157797, 0.00167693, ..., 0.00229663, 0.00066319,
        0.00171414]])

In [25]:
for seed in range(nstarts):
    print(f'Train seed {seed}')
    set_seed(seed)
    seed_oof = []
    seed_targets = []
    seed_preds = np.zeros((len(test), ntargets, nfolds))
    
    for n in range(nfolds):
        print(f'Train fold {n+1}')
        xtrain, xval = train[folds!=n], train[folds==n]
        ytrain, yval = train_targets[folds!=n], train_targets[folds==n]
        xtrain = np.concatenate([xtrain, train_pl], axis=0)
        ytrain = np.concatenate([ytrain, train_targets_pl], axis=0)
        
        tabnet_params = dict(
              n_d = 32,
              n_a = 32,
              n_steps = 1,
              gamma = 1.3,
              lambda_sparse = 0,
              optimizer_fn = optim.Adam,
              optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
              mask_type = "entmax",
              scheduler_params = dict(mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
              scheduler_fn = ReduceLROnPlateau,
              #seed = 42,
              verbose = 10
                     )
        model = TabNetRegressor(**tabnet_params, seed=seed)
        model.fit(
            X_train = xtrain,
            y_train = ytrain,
            eval_set = [(xval, yval)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs = nepochs,
            patience = 30,
            batch_size = 1024,
            virtual_batch_size = 32,
            num_workers = 1,
            drop_last = False,
            loss_fn = trn_criterion
        )
        #######################
        #### SAVE WEIGHTS #####
        #######################
        print('Saving weights...')
        path = f"tabnet fold{n}_seed{seed}"
        model.save_model(path)
        print('Weights saved!')
        
        ##################
        ### INFERENCE ####
        ##################
        #print('Loading weights...')
        #model.load_model(path)
        fold_oof = model.predict(xval)
        fold_oof = 1 / (1 + np.exp(-fold_oof))
        fold_preds = model.predict(test)
        fold_preds = 1 / (1 + np.exp(-fold_preds))
        
        seed_oof.append(fold_oof)
        seed_targets.append(yval)
        seed_preds[:, :, n] = fold_preds
    
    seed_oof = np.concatenate(seed_oof)
    seed_targets = np.concatenate(seed_targets)
    seed_preds = np.mean(seed_preds, axis=2)
    
    print("Score for this seed {:5.5f}".format(mean_log_loss(seed_targets, seed_oof)))
    oof_targets = seed_targets
    oof[:, seed, :] = seed_oof
    preds += seed_preds / nstarts

oof = np.mean(oof, axis=1)
print("Overall score is {:5.5f}".format(mean_log_loss(oof_targets, oof)))       

Train seed 0
Train fold 1
Device used : cuda
epoch 0  | loss: 0.31312 | val_logits_ll: 0.02732 |  0:00:02s
epoch 10 | loss: 0.02267 | val_logits_ll: 0.0193  |  0:00:19s
epoch 20 | loss: 0.02128 | val_logits_ll: 0.01847 |  0:00:36s
epoch 30 | loss: 0.02095 | val_logits_ll: 0.01792 |  0:00:55s
epoch 40 | loss: 0.02079 | val_logits_ll: 0.0177  |  0:01:12s
epoch 50 | loss: 0.02056 | val_logits_ll: 0.01758 |  0:01:29s
epoch 60 | loss: 0.02034 | val_logits_ll: 0.0176  |  0:01:46s
epoch 70 | loss: 0.02019 | val_logits_ll: 0.01769 |  0:02:05s
epoch 80 | loss: 0.01999 | val_logits_ll: 0.01773 |  0:02:22s
epoch 90 | loss: 0.01987 | val_logits_ll: 0.01746 |  0:02:39s
epoch 100| loss: 0.01964 | val_logits_ll: 0.01758 |  0:02:58s
epoch 110| loss: 0.01942 | val_logits_ll: 0.01752 |  0:03:15s

Early stopping occured at epoch 114 with best_epoch = 84 and best_val_logits_ll = 0.01732
Best weights from best epoch are automatically used!
Saving weights...
Successfully saved model at tabnet fold0_seed0.zi

In [26]:
print("Overall score is {:5.5f}".format(mean_log_loss(oof_targets, oof)))       

Overall score is 0.01698


In [27]:
np.save('tabnet_oof.npy', oof)
np.save('tabnet_targets.npy', oof_targets)

In [28]:
ss[targets] = preds
ss.loc[test_features['cp_type']=='ctl_vehicle', targets] = 0
ss.to_csv('submission.csv', index=False)